In [3]:
!ls /workspace/results


dataflow_partition0_kjxndb47		    end2end_quartznet_tidy.onnx
end2end_quartznet_dataflow_partition.onnx   exec_output
end2end_quartznet_export_dev.onnx	    input_sample_float_0.npy
end2end_quartznet_lowered.onnx		    partitioning_lowering
end2end_quartznet_lowered_partitioned.onnx  partitioning_repartition
end2end_quartznet_streamline.onnx	    quartznet_pre_hls.npy


In [1]:
from finn.util.visualization import showInNetron

showInNetron("/workspace/results/end2end_quartznet_lowered_partitioned.onnx")

Serving '/workspace/results/end2end_quartznet_lowered_partitioned.onnx' at http://0.0.0.0:8081


In [5]:
from finn.transformation.general import RemoveUnusedTensors
from finn.transformation.extend_partition import ExtendPartition
import finn.core.data_layout as DataLayout
from finn.transformation.infer_shapes import InferShapes
from finn.transformation.infer_data_layouts import InferDataLayouts
from finn.core.modelwrapper import ModelWrapper

model = ModelWrapper("/workspace/results/end2end_quartznet_lowered_partitioned.onnx")

list_of_nodes = [n for n in model.graph.node]
for n in list_of_nodes:
    #if n.name in ["GenericPartition_0", "GenericPartition_1", "GenericPartition_2"]:
    if n.name in ["GenericPartition_3", "GenericPartition_4"]:
        if n.name=="GenericPartition_3":
            global_in = n.input[0]
            global_in_vi = model.get_tensor_valueinfo(global_in)
            model.graph.input.insert(0, global_in_vi)
            model.graph.input.pop(1)
        if n.name=="GenericPartition_4":
            global_out = n.output[0]
            global_out_vi = model.get_tensor_valueinfo(global_out)
            model.graph.output.insert(0, global_out_vi)
            model.graph.output.pop(1)
        continue
    model.graph.node.remove(n)
    
model = model.transform(RemoveUnusedTensors())
model = model.transform(ExtendPartition([0, 1]))

## ONLY when generic_partition0 is unfolded
# Set tensor layout of input to first Conv and MT to NCHW
#in_conv = model.graph.node[0].input[0]
#in_mt = model.graph.node[1].input[0]
#for t in [in_conv, in_mt]:
#    model.set_tensor_layout(t, DataLayout.NCHW)


model = model.transform(InferShapes())
model = model.transform(InferDataLayouts())
    
model.save("/tmp/pre_hls_partition34.onnx")

In [7]:
from finn.core.modelwrapper import ModelWrapper
from finn.util.visualization import showInNetron

showInNetron("/tmp/pre_hls_partition34.onnx")


Stopping http://0.0.0.0:8081
Serving '/tmp/pre_hls_partition34.onnx' at http://0.0.0.0:8081


In [ ]:
from finn.core.modelwrapper import ModelWrapper
from finn.custom_op.registry import getCustomOp
import finn.transformation.fpgadataflow.convert_to_hls_layers as to_hls
from finn.transformation.fpgadataflow.prepare_cppsim import PrepareCppSim
from finn.transformation.fpgadataflow.compile_cppsim import CompileCppSim
from finn.transformation.fpgadataflow.set_exec_mode import SetExecMode
from finn.transformation.fpgadataflow.set_folding import SetFolding
from finn.util.fpgadataflow import is_fpgadataflow_node
from finn.transformation.streamline.absorb import AbsorbConsecutiveTransposes
from finn.transformation.general import GiveUniqueNodeNames
from finn.transformation.fpgadataflow.prepare_ip import PrepareIP
from finn.transformation.fpgadataflow.hlssynth_ip import HLSSynthIP
from finn.transformation.fpgadataflow.prepare_rtlsim import PrepareRTLSim

exec_mode="cppsim"

model = ModelWrapper("/tmp/pre_hls_partition34.onnx")
model = model.transform(to_hls.InferConvInpGen(), make_deepcopy=False)
model = model.transform(to_hls.InferVVAU(), make_deepcopy=False)
model = model.transform(to_hls.InferQuantizedStreamingFCLayer("decoupled"), make_deepcopy=False)
model = model.transform(to_hls.InferThresholdingLayer(), make_deepcopy=False)
model = model.transform(to_hls.InferAddStreamsLayer(), make_deepcopy=False)
model = model.transform(to_hls.InferDuplicateStreamsLayer(), make_deepcopy=False)

# Two consecutive transposes that can be absorbed:
#model = model.transform(AbsorbConsecutiveTransposes())

for n in model.graph.node:
    if is_fpgadataflow_node(n):
        inst = getCustomOp(n)
        if n.op_type == "DuplicateStreams_Batch":
            inst.set_nodeattr("PE", 1)
        elif n.op_type=="FMPadding_Batch":
            inst.set_nodeattr("SIMD", 1)
        elif n.op_type=="AddStreams_Batch":
            inst.set_nodeattr("PE", 1)
        elif n.op_type=="ConvolutionInputGenerator1D":
            inst.set_nodeattr("ram_style", "distributed")
            inst.set_nodeattr("SIMD", 1)
        elif n.op_type=="Vector_Vector_Activate_Batch":
            inst.set_nodeattr("resType", "dsp")
            inst.set_nodeattr("PE", 1)
        elif n.op_type=="StreamingFCLayer_Batch":
            inst.set_nodeattr("resType", "dsp")
            inst.set_nodeattr("ram_style", "ultra")
            inst.set_nodeattr("mem_mode", "decoupled")
            if inst.get_nodeattr("ram_style")=="ultra":
                inst.set_nodeattr("runtime_writeable_weights", 1)
            inst.set_nodeattr("PE", 1)
            inst.set_nodeattr("SIMD", 2)
        elif n.op_type=="Thresholding_Batch":
            inst.set_nodeattr("ram_style", "distributed")
            inst.set_nodeattr("mem_mode", "const")
            inst.set_nodeattr("PE", 1)
        else:
            print("Missed: {} in folding!".format(n.op_type))
            break

if exec_mode=="cppsim":
    model = model.transform(PrepareCppSim())
    model = model.transform(CompileCppSim())
    model = model.transform(SetExecMode("cppsim"))
    model = model.transform(GiveUniqueNodeNames())
elif exec_mode == "rtlsim":
    model = model.transform(SetExecMode("rtlsim"))
    model = model.transform(GiveUniqueNodeNames())
    model = model.transform(PrepareIP("xc7z020clg400-1", 5))
    model = model.transform(HLSSynthIP())
    model = model.transform(PrepareRTLSim())

model.save("/tmp/hls_partition34.onnx")


/workspace/finn/src/finn/custom_op/fpgadataflow/vector_vector_activate_batch.py:72: UserWarning: Clipping some thresholds in 
  warnings.warn("Clipping some thresholds in %s" % self.onnx_node.name)
/workspace/finn/src/finn/custom_op/fpgadataflow/streamingfclayer_batch.py:589: UserWarning: Clipping some thresholds in 
  warnings.warn("Clipping some thresholds in %s" % self.onnx_node.name)
/workspace/finn/src/finn/custom_op/fpgadataflow/thresholding_batch.py:137: UserWarning: inputDataType changing for : DataType.INT32 -> DataType.INT5 
  warnings.warn(warn_str)


In [15]:
from finn.util.visualization import showInNetron

#showInNetron("/tmp/pre_hls.onnx")
#showInNetron("/tmp/hls_without_threshold.onnx")
showInNetron("/tmp/hls_partition123.onnx")

Stopping http://0.0.0.0:8081
Serving '/tmp/hls_partition123.onnx' at http://0.0.0.0:8081


In [10]:
showInNetron("/workspace/results/partitioning_repartition/partition_0.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/results/partitioning_repartition/partition_0.onnx' at http://0.0.0.0:8081


# Execute & save

In [20]:
import time 
import numpy as np
from finn.core.datatype import DataType
import finn.core.onnx_exec as oxe
from finn.core.modelwrapper import ModelWrapper

step_name = "pre_hls_partition123"
model = ModelWrapper("/tmp/pre_hls_partition123.onnx")

#step_name = "post_hls_partition123"
#model = ModelWrapper("/tmp/hls_partition123.onnx")

s_time = time.time()

iname = model.graph.input[0].name
oname = model.graph.output[0].name
inp_val = np.load("/workspace/results/input_sample_float_0.npy")
inp_val = np.reshape(inp_val, np.shape(inp_val)+(1,)) # make 4D tensor
input_dict = {iname: inp_val}
# Change tensor annotation to FLOAT for simulation (must be kept INT otherwise InferDataTypes will annotate all tensors as FLOAT)
model = model.transform(ExtendPartition([0]))
first_conv_out = model.graph.node[0].output[0]
#first_transpose_out = model.graph.node[1].output[0]
model.set_tensor_datatype(iname, DataType.FLOAT32)
model.set_tensor_datatype(first_conv_out, DataType.FLOAT32)
#model.set_tensor_datatype(first_transpose_out, DataType.FLOAT32)
# Execute FINN simulation
output_dict = oxe.execute_onnx(model, input_dict, return_full_exec_context=True)
produced = output_dict[oname]

## Save
np.save("/workspace/results/quartznet_"+step_name+".npy", output_dict)

e_time = time.time()

print("TIME: {}".format(e_time-s_time))


Conv_0
MultiThreshold_0


MultiThreshold_1



MultiThreshold_3
MultiThreshold_2

MultiThreshold_4


MultiThreshold_5

MultiThreshold_6


MultiThreshold_7

MultiThreshold_8


MultiThreshold_9

MultiThreshold_10


MultiThreshold_11

MultiThreshold_12
Add_0
MultiThreshold_13



MultiThreshold_15
MultiThreshold_14



/workspace/finn-base/src/finn/util/basic.py:377: UserWarning: The values of tensor MultiThreshold_16_param0 can't be represented with the set FINN datatype (DataType.INT32), they will be rounded to match the FINN datatype.
  warnings.warn(


MultiThreshold_16




/workspace/finn-base/src/finn/util/basic.py:377: UserWarning: The values of tensor MultiThreshold_17_param0 can't be represented with the set FINN datatype (DataType.INT32), they will be rounded to match the FINN datatype.
  warnings.warn(


MultiThreshold_17



...

MultiThreshold_21



/workspace/finn-base/src/finn/util/basic.py:377: UserWarning: The values of tensor MultiThreshold_22_param0 can't be represented with the set FINN datatype (DataType.INT32), they will be rounded to match the FINN datatype.
  warnings.warn(


MultiThreshold_22




/workspace/finn-base/src/finn/util/basic.py:377: UserWarning: The values of tensor MultiThreshold_23_param0 can't be represented with the set FINN datatype (DataType.INT32), they will be rounded to match the FINN datatype.
  warnings.warn(


MultiThreshold_23

MultiThreshold_24
Add_1
TIME: 687.5375733375549


# Comparison of outputs


## Pre vs Post

In [23]:
import numpy as np

pre_hls = np.load("/workspace/results/quartznet_pre_hls.npy", allow_pickle=True).tolist()

post_hls = np.load("/workspace/results/quartznet_post_hls.npy", allow_pickle=True).tolist()

for k,v in pre_hls.items():
    if k=="Conv_0_out0":
        np.save("/tmp/conv_0_out0_pre_hls.npy", v)
        is_equal = (v==post_hls[k]).all()
        print("{}: {}".format(k,is_equal))
    if k=="MultiThreshold_0_out0":
        is_equal = (v==post_hls[k]).all()
        wrong_array = v-post_hls[k]
        print("{}: {}".format(k,is_equal))
    if k=="ywWyGd":
        is_equal = (v==post_hls[k]).all()
        wrong_array = v-post_hls[k]
        print("{}: {}".format(k,is_equal))
    if k=="fOrrGG":
        is_equal = (v==post_hls[k]).all()
        #wrong_array = v-post_hls[k]
        print("{}: {}".format(k,is_equal))
    if k=="MultiThreshold_12_out0":
        is_equal = (v==post_hls[k]).all()
        wrong_array = v-post_hls[k]
        print("{}: {}".format(k,is_equal))

MultiThreshold_12_out0: False
ywWyGd: False
Conv_0_out0: True
MultiThreshold_0_out0: False
fOrrGG: False


In [24]:
for el in wrong_array.flatten():
    print(el)

-1.0
-1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-1.0
0.0
0.0
-1.0
-1.0
0.0
0.0
0.0
-1.0
0.0
0.0
0.0
0.0
0.0
-1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-1.0
0.0
0.0
0.0
0.0
0.0
-1.0
0.0
0.0
0.0
0.0
0.0
0.0
-1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-1.0
-1.0
0.0
0.0
0.0
-1.0
-1.0
-1.0
0.0
-1.0
-1.0
0.0
0.0
0.0
-1.0
0.0
0.0
-1.0
-1.0
0.0
0.0
0.0
0.0
-1.0
0.0
-1.0
-1.0
-1.0
0.0
0.0
-1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-1.0
-1.0
-1.0
-1.0
-1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-1.0
-1.0
0.0
-1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-1.0
-1.0
-1.0
-1.0
0.0
-1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-1.0
0.0
0

-1.0
-1.0
0.0
0.0
0.0
0.0
0.0
-1.0
0.0
0.0
-1.0
0.0
0.0
0.0
-1.0
0.0
0.0
0.0
0.0
0.0
0.0
-1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-1.0
-1.0
0.0
-1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-1.0
0.0
-1.0
0.0
0.0
-1.0
-1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-1.0
0.0
0.0
0.0
0.0
-1.0
0.0
0.0
-1.0
-1.0
0.0
-1.0
-1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-1.0
0.0
0.0
0.0
0.0
0.0
-1.0
-1.0
0.0
0.0
0.0
0.0
-1.0
0.0
-1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-1.0
-1.0
0.0
-1.0
0.0
-1.0
0.0
-1.0
0.0
0.0
0.0
0.0
0.0
-1.0
0.0
-1.0
0.0
0.0
-1.0
0.0
0.0
0.0
-1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-1.0
-1.0
0.0
-1.0
0.0
0.0
0.0
0.0
0.0
0.0
-1.0
0.0
0.0
0.0
0.0
0.0
0.0
-1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-1.0
0.0
-1.0
0.0
0.0
-1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-1.0
-1.0
0.0


-1.0
0.0
-1.0
0.0
0.0
0.0
0.0
-1.0
0.0
0.0
0.0
0.0
0.0
-1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-1.0
0.0
0.0
0.0
-1.0
0.0
0.0
0.0
-1.0
0.0
-1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-1.0
0.0
0.0
0.0
0.0
-1.0
0.0
0.0
0.0
0.0
-1.0
0.0
-1.0
0.0
0.0
0.0
0.0
0.0
-1.0
0.0
-1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-1.0
0.0
0.0
0.0
-1.0
0.0
0.0
0.0
-1.0
0.0
0.0
0.0
0.0
-1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-1.0
0.0
0.0
0.0
-1.0
0.0
0.0
0.0
-1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


## Pre vs post 123 partition

In [22]:
!ls /workspace/results

dataflow_partition0_kjxndb47
end2end_quartznet_dataflow_partition.onnx
end2end_quartznet_export_dev.onnx
end2end_quartznet_lowered.onnx
end2end_quartznet_lowered_partitioned.onnx
end2end_quartznet_streamline.onnx
end2end_quartznet_tidy.onnx
exec_output
input_sample_float_0.npy
partitioning_lowering
partitioning_repartition
quartznet_post_hls.npy
quartznet_post_hls_partition123.npy
quartznet_post_hls_without_threshold.npy
quartznet_pre_hls.npy
quartznet_pre_hls_partition123.npy


In [26]:
import numpy as np

pre_hls = np.load("/workspace/results/quartznet_pre_hls_partition123.npy", allow_pickle=True).tolist()

post_hls = np.load("/workspace/results/quartznet_post_hls_partition123.npy", allow_pickle=True).tolist()

for k,v in pre_hls.items():
    ## ONNX part
    if k=="Conv_0_out0":
        is_equal = (v==post_hls[k]).all()
        print("{}: {}".format(k,is_equal))
    if k=="MultiThreshold_0_out0":
        is_equal = (v==post_hls[k]).all()
        wrong_array = v-post_hls[k]
        print("{}: {}".format(k,is_equal))
    ## Before first partition
    if k=="ywWyGd":
        is_equal = (v==post_hls[k]).all()
        wrong_array = v-post_hls[k]
        print("{}: {}".format(k,is_equal))
    if k=="fOrrGG":
        is_equal = (v==post_hls[k]).all()
        #wrong_array = v-post_hls[k]
        print("{}: {}".format(k,is_equal))
    # First partition output
    if k=="MultiThreshold_12_out0":
        is_equal = (v==post_hls[k]).all()
        wrong_array = v-post_hls[k]
        print("{}: {}".format(k,is_equal))
    if k=="HxvEWO":
        is_equal = (v==post_hls[k]).all()
        wrong_array = v-post_hls[k]
        print("{}: {}".format(k, is_equal))
    # Second partition output
    if k=="MultiThreshold_24_out0":
        is_equal = (v==post_hls[k]).all()
        wrong_array = v-post_hls[k]
        print("{}: {}".format(k,is_equal))

MultiThreshold_24_out0: True
ywWyGd: True
MultiThreshold_12_out0: True
fOrrGG: True
HxvEWO: True
Conv_0_out0: True
MultiThreshold_0_out0: True


In [27]:
np.save("/workspace/results/input_partition45.npy", pre_hls["MultiThreshold_24_out0"])

In [12]:
import numpy as np

labels = [" ", "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m",
         "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z", "'"]
    
def __ctc_decoder_predictions_tensor(tensor, labels):
    """
    Decodes a sequence of labels to words
    """
    blank_id = len(labels)
    hypotheses = []
    labels_map = dict([(i, labels[i]) for i in range(len(labels))])
    decoded_prediction = []
    previous = len(labels)  # id of a blank symbol
    prediction = tensor
    for p in prediction:
        if (p != previous or previous == blank_id) and p != blank_id:
            decoded_prediction.append(p)
        previous = p
    hypothesis = ''.join([labels_map[c] for c in decoded_prediction])
    hypotheses.append(hypothesis)
    return hypotheses


output = np.load("/workspace/results/results_qce/quartznet_cppsim.npy")
print(__ctc_decoder_predictions_tensor(output.flatten(), labels))


['vanity and vexation of spat']


# Start from scratch... takes too long to run execution (>4 hours..)

In [3]:
import time
import os
import json
import pathlib
import pytest
import numpy as np

import finn.core.onnx_exec as oxe
from finn.transformation.fold_constants import FoldConstants
from finn.transformation.general import RemoveStaticGraphInputs
from finn.transformation.infer_shapes import InferShapes
#import brevitas_examples.speech_to_text as stt

from finn.custom_op.registry import getCustomOp
from finn.util.test import (
    load_test_checkpoint_or_skip
)
from finn.core.modelwrapper import ModelWrapper
from finn.core.datatype import DataType
from finn.util.basic import get_by_name

from finn.transformation.change_3d_tensors_to_4d import Change3DTo4DTensors
from finn.transformation.general import (
    GiveUniqueNodeNames,
    GiveRandomTensorNames,
    GiveReadableTensorNames,
    GiveUniqueParameterTensors
)
from finn.transformation.batchnorm_to_affine import BatchNormToAffine
from finn.transformation.streamline.reorder import (
    MoveAddPastMul,
    MoveAddPastConv,
    MoveMulPastFork,
    MoveScalarMulPastConv,
    MoveMulPastDWConv,
    MoveLinearPastEltwiseAdd
)
from finn.transformation.streamline.collapse_repeated import(
    CollapseRepeatedAdd,
    CollapseRepeatedMul
)
from finn.transformation.streamline.absorb import(
    AbsorbAddIntoMultiThreshold,
    AbsorbMulIntoMultiThreshold,
    FactorOutMulSignMagnitude,
    Absorb1BitMulIntoConv,
    AbsorbSignBiasIntoMultiThreshold
)

from finn.transformation.streamline.round_thresholds import RoundAndClipThresholds
from finn.transformation.infer_datatypes import InferDataTypes
from finn.transformation.create_generic_partitions import PartitionFromDict
from finn.transformation.lower_convs_to_matmul import LowerConvsToMatMul
from finn.transformation.streamline.absorb import AbsorbTransposeIntoMultiThreshold
from finn.transformation.streamline.reorder import (
    MoveTransposePastMultiThreshold,
    MoveTransposePastJoinAdd,
    MoveTransposeBeforeFork
)
from finn.transformation.extend_partition import ExtendPartition
import finn.transformation.fpgadataflow.convert_to_hls_layers as to_hls
from finn.transformation.fpgadataflow.create_dataflow_partition import CreateDataflowPartition
from finn.transformation.fpgadataflow.prepare_cppsim import PrepareCppSim
from finn.transformation.fpgadataflow.compile_cppsim import CompileCppSim
from finn.transformation.fpgadataflow.set_exec_mode import SetExecMode
from finn.transformation.fpgadataflow.prepare_ip import PrepareIP
from finn.transformation.fpgadataflow.hlssynth_ip import HLSSynthIP
from finn.transformation.fpgadataflow.replace_verilog_relpaths import (
    ReplaceVerilogRelPaths,
)
from finn.transformation.fpgadataflow.annotate_resources import AnnotateResources
from finn.transformation.fpgadataflow.set_folding import SetFolding
from finn.util.basic import alveo_part_map, alveo_default_platform
from finn.util.config import extract_model_config_to_json
from finn.transformation.fpgadataflow.set_fifo_depths import InsertAndSetFIFODepths
from finn.transformation.fpgadataflow.create_stitched_ip import CreateStitchedIP
from finn.transformation.fpgadataflow.annotate_cycles import AnnotateCycles
from finn.analysis.fpgadataflow.exp_cycles_per_layer import exp_cycles_per_layer
from finn.analysis.fpgadataflow.res_estimation import (
    res_estimation,
    res_estimation_complete
)
from finn.analysis.fpgadataflow.op_and_param_counts import (
    aggregate_dict_keys,
    op_and_param_counts
)
from finn.analysis.fpgadataflow.dataflow_performance import dataflow_performance
from finn.analysis.fpgadataflow.hls_synth_res_estimation import hls_synth_res_estimation
from finn.transformation.fpgadataflow.prepare_rtlsim import PrepareRTLSim
from finn.core.throughput_test import throughput_test_rtlsim
from copy import deepcopy


def test_end2end_quartznet_tidy_and_change_shape_tensors(verify=False):
    model = ModelWrapper("/workspace/results/end2end_quartznet_export_dev.onnx")

    model = model.transform(GiveUniqueNodeNames())
    model = model.transform(GiveRandomTensorNames())
    model = model.transform(GiveReadableTensorNames())
    model = model.transform(GiveUniqueParameterTensors())

    # Convert to supported format
    model = model.transform(Change3DTo4DTensors())

    model.save("/workspace/results/end2end_quartznet_tidy.onnx")

def test_end2end_quartznet_streamline(verify=False):
    model = ModelWrapper("/workspace/results/end2end_quartznet_tidy.onnx")

    # Absorb sign bias from export into MultiThreshold node
    model = model.transform(AbsorbSignBiasIntoMultiThreshold())
    # Collapse BatchNorm to Add and Mul
    model = model.transform(BatchNormToAffine())
    # Group multiplications
    model = model.transform(MoveMulPastFork())
    model = model.transform(MoveScalarMulPastConv())
    model = model.transform(MoveMulPastDWConv())
    # Move Mul/Add past join node
    model = model.transform(MoveLinearPastEltwiseAdd())
    # Collapes additions & multiplications
    model = model.transform(CollapseRepeatedAdd())
    model = model.transform(CollapseRepeatedMul())
    # Absorb Add/Mul into multithreshold
    model = model.transform(AbsorbAddIntoMultiThreshold())
    model = model.transform(FactorOutMulSignMagnitude())
    model = model.transform(Absorb1BitMulIntoConv())
    model = model.transform(AbsorbMulIntoMultiThreshold())

    # Ensure thresholds are integers
    ## Add quantization annotation to ensure RoundAndClipThresholds works
    global_input_name = model.graph.input[0].name
    model.set_tensor_datatype(global_input_name, DataType.INT8)
    model = model.transform(InferDataTypes())
    model = model.transform(RoundAndClipThresholds())

    # Remove floating point scalar multiplication before argmax
    mul_nodes = [x for x in model.graph.node if (x.op_type=="Mul")]
    for n_mul in mul_nodes:
        input_mul = n_mul.input[0]
        node_after_mul = model.find_consumer(n_mul.output[0])
        node_after_mul.input[0] = input_mul
        model.graph.node.remove(n_mul)

    model = model.transform(GiveUniqueNodeNames())
    model = model.transform(GiveRandomTensorNames())
    model = model.transform(GiveReadableTensorNames())
    model = model.transform(GiveUniqueParameterTensors())

    model.save("/workspace/results/end2end_quartznet_streamline.onnx")

def test_end2end_quartznet_lowering(verify=False):
    model = ModelWrapper("/workspace/results/end2end_quartznet_streamline.onnx")
    partitionings = {1: range(2, 75),
                    2: range(75, 147),
                    3: range(147, 219),
                    4: range(219, 291),
                    5: range(291, 363),
                    6: range(363, 375)}
    model = model.transform(PartitionFromDict(partitionings, "/workspace/results/partitioning_lowering"))

    for n in model.graph.node:
        if n.op_type=="GenericPartition":
            path_to_partition = get_by_name(n.attribute, "model", "name").s.decode('utf-8')
            model_partition = ModelWrapper(path_to_partition)
            # Lowering
            model_partition = model_partition.transform(LowerConvsToMatMul())
            # Absorb transpose node
            model_partition = model_partition.transform(AbsorbTransposeIntoMultiThreshold())
            # Reorder remaining transpose nodes
            model_partition = model_partition.transform(MoveTransposePastMultiThreshold())
            model_partition = model_partition.transform(MoveTransposePastJoinAdd())
            model_partition = model_partition.transform(MoveTransposeBeforeFork())

            model_partition.save(path_to_partition)

    model.save("/workspace/results/end2end_quartznet_lowered.onnx")


def test_end2end_quartznet_repartition(verify=False):
    model = ModelWrapper("/workspace/results/end2end_quartznet_lowered.onnx")
    partitionings = [{0: range(0,3), 1: range(3, 33), 2: range(33, 62), 3: range(62, 91), 4: range(91, 120), 5: range(120, 149), 6: range(149, 178)},
                     {7: range(7, 36), 8: range(36, 65), 9: range(65, 94)},
                     {10: range(10, 39), 11: range(39, 68), 12: range(68, 97)},
                     {13: range(13, 42), 14: range(42, 71), 15: range(71, 100)},
                     {16: range(16, 25)}
                    ]

    nodes = [n for n in model.graph.node if n.op_type=="GenericPartition"]
    for ind, n in enumerate(nodes):
        if ind == 0:
            node_ind_to_unfold = [2+ind, 2+ind+1] # unfold current and next node
        else:
            node_ind_to_unfold = [3*ind+5] # (+1 for initial partition, +1 transpose node, +3 partitions)

        model = model.transform(ExtendPartition(node_ind_to_unfold))
        model = model.transform(AbsorbTransposeIntoMultiThreshold())

        if ind==0:
            model = model.transform(PartitionFromDict(partitionings[0], "/workspace/results/partitioning_repartition"))
        if ind==1:
            model = model.transform(PartitionFromDict(partitionings[1], "/workspace/results/partitioning_repartition"))
        if ind==2:
            model = model.transform(PartitionFromDict(partitionings[2], "/workspace/results/partitioning_repartition"))
        if ind==3:
            model = model.transform(PartitionFromDict(partitionings[3], "/workspace/results/partitioning_repartition"))
        if ind==4:
            model = model.transform(PartitionFromDict(partitionings[4], "/workspace/results/partitioning_repartition"))
            break

    model = model.transform(GiveUniqueNodeNames())
    model.save("/workspace/results/end2end_quartznet_lowered_partitioned.onnx")

    
def test_end2end_quartznet_convert_to_hls_layers():
    model = ModelWrapper("/workspace/results/end2end_quartznet_lowered_partitioned.onnx")

    partition_dir = "/workspace/results/partitioning_hls"
    partition_id = 1
    nodes = [n for n in model.graph.node if (n.op_type=="GenericPartition" and n.name!="GenericPartition_0")]
    for n in nodes:
        #inst = GetCustomOp(n)
        prefix = "pt_"+str(partition_id)+"_"

        path_to_partition = get_by_name(n.attribute, "model", "name").s.decode('utf-8')
        model_partition = ModelWrapper(path_to_partition)

        model_partition = model_partition.transform(to_hls.InferConvInpGen(), make_deepcopy=False)
        #model_partition = model_partition.transform(to_hls.InferVVAU(), make_deepcopy=False)
        #model_partition = model_partition.transform(to_hls.InferQuantizedStreamingFCLayer(mem_mode), make_deepcopy=False)
        #model_partition = model_partition.transform(to_hls.InferThresholdingLayer(), make_deepcopy=False)
        #model_partition = model_partition.transform(to_hls.InferAddStreamsLayer(), make_deepcopy=False)
        #model_partition = model_partition.transform(to_hls.InferDuplicateStreamsLayer(), make_deepcopy=False)

        pathlib.Path(self.partition_dir).mkdir(parents=True, exist_ok=True)
        partition_path = partition_dir+"/partition_"+str(partition_id)+".onnx"
        model_partition.save(path_to_partition)
        inst.set_nodeattr("model", partition_path)

        partition_id+=1

    model.save("/workspace/results/end2end_quartznet_hls_layers.onnx")
    
    

In [4]:
test_end2end_quartznet_tidy_and_change_shape_tensors()
test_end2end_quartznet_streamline()
test_end2end_quartznet_lowering()
test_end2end_quartznet_repartition()
#test_end2end_quartznet_convert_to_hls_layers()

In [19]:
!ls /workspace/results

end2end_quartznet_export_dev.onnx	    end2end_quartznet_tidy.onnx
end2end_quartznet_lowered.onnx		    input_sample_float_0.npy
end2end_quartznet_lowered_partitioned.onnx  partitioning_lowering
end2end_quartznet_streamline.onnx	    partitioning_repartition


In [18]:
step_name = "lowered_partitioned"
model = ModelWrapper("/workspace/results/end2end_quartznet_"+step_name+".onnx")

s_time = time.time()

iname = model.graph.input[0].name
oname = model.graph.output[0].name
inp_val = np.load("/workspace/results/input_sample_float_0.npy")
inp_val = np.reshape(inp_val, np.shape(inp_val)+(1,)) # make 4D tensor
input_dict = {iname: inp_val}
# Change tensor annotation to FLOAT for simulation (must be kept INT otherwise InferDataTypes will annotate all tensors as FLOAT)
model = model.transform(ExtendPartition([0]))
first_conv_out = model.graph.node[0].output[0]
model.set_tensor_datatype(iname, DataType.FLOAT32)
model.set_tensor_datatype(first_conv_out, DataType.FLOAT32)
# Execute FINN simulation
output_dict = oxe.execute_onnx(model, input_dict)
produced = output_dict[oname]
np.save("/workspace/results/quartznet_"+step_name+".npy", produced)

e_time = time.time()

print("TIME: {}".format(e_time-s_time))


Conv_0_out0


/workspace/finn-base/src/finn/util/basic.py:377: UserWarning: The values of tensor MultiThreshold_4_param0 can't be represented with the set FINN datatype (DataType.INT32), they will be rounded to match the FINN datatype.
  warnings.warn(
/workspace/finn-base/src/finn/util/basic.py:377: UserWarning: The values of tensor MultiThreshold_5_param0 can't be represented with the set FINN datatype (DataType.INT32), they will be rounded to match the FINN datatype.
  warnings.warn(
/workspace/finn-base/src/finn/util/basic.py:377: UserWarning: The values of tensor MultiThreshold_6_param0 can't be represented with the set FINN datatype (DataType.INT32), they will be rounded to match the FINN datatype.
  warnings.warn(
/workspace/finn-base/src/finn/util/basic.py:377: UserWarning: The values of tensor MultiThreshold_7_param0 can't be represented with the set FINN datatype (DataType.INT32), they will be rounded to match the FINN datatype.
  warnings.warn(
/workspace/finn-base/src/finn/util/basic.py:

/workspace/finn-base/src/finn/util/basic.py:377: UserWarning: The values of tensor MultiThreshold_56_param0 can't be represented with the set FINN datatype (DataType.INT32), they will be rounded to match the FINN datatype.
  warnings.warn(
/workspace/finn-base/src/finn/util/basic.py:377: UserWarning: The values of tensor MultiThreshold_57_param0 can't be represented with the set FINN datatype (DataType.INT32), they will be rounded to match the FINN datatype.
  warnings.warn(
/workspace/finn-base/src/finn/util/basic.py:377: UserWarning: The values of tensor MultiThreshold_58_param0 can't be represented with the set FINN datatype (DataType.INT32), they will be rounded to match the FINN datatype.
  warnings.warn(
/workspace/finn-base/src/finn/util/basic.py:377: UserWarning: The values of tensor MultiThreshold_59_param0 can't be represented with the set FINN datatype (DataType.INT32), they will be rounded to match the FINN datatype.
  warnings.warn(
/workspace/finn-base/src/finn/util/basic

/workspace/finn-base/src/finn/util/basic.py:377: UserWarning: The values of tensor MultiThreshold_107_param0 can't be represented with the set FINN datatype (DataType.INT32), they will be rounded to match the FINN datatype.
  warnings.warn(
/workspace/finn-base/src/finn/util/basic.py:377: UserWarning: The values of tensor MultiThreshold_108_param0 can't be represented with the set FINN datatype (DataType.INT32), they will be rounded to match the FINN datatype.
  warnings.warn(
/workspace/finn-base/src/finn/util/basic.py:377: UserWarning: The values of tensor MultiThreshold_112_param0 can't be represented with the set FINN datatype (DataType.INT32), they will be rounded to match the FINN datatype.
  warnings.warn(
/workspace/finn-base/src/finn/util/basic.py:377: UserWarning: The values of tensor MultiThreshold_113_param0 can't be represented with the set FINN datatype (DataType.INT32), they will be rounded to match the FINN datatype.
  warnings.warn(
/workspace/finn-base/src/finn/util/b

KeyboardInterrupt: 

In [6]:
labels = [" ", "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m",
         "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z", "'"]
    
def __ctc_decoder_predictions_tensor(tensor, labels):
    """
    Decodes a sequence of labels to words
    """
    blank_id = len(labels)
    hypotheses = []
    labels_map = dict([(i, labels[i]) for i in range(len(labels))])
    decoded_prediction = []
    previous = len(labels)  # id of a blank symbol
    prediction = tensor
    for p in prediction:
        if (p != previous or previous == blank_id) and p != blank_id:
            decoded_prediction.append(p)
        previous = p
    hypothesis = ''.join([labels_map[c] for c in decoded_prediction])
    hypotheses.append(hypothesis)
    return hypotheses


output = np.load("/workspace/results/quartznet_"+step_name+".npy")
print(__ctc_decoder_predictions_tensor(output.flatten(), labels))


NameError: name 'step_name' is not defined